In [1]:
require("/home/amit/Downloads/SemVII/QuDynamics.jl/src/QuDynamics.jl")

 in depwarn at deprecated.jl:73
 [inlined code] from deprecated.jl:694
 in require at no file:0
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/amit/.julia/v0.4/IJulia/src/execute_request.jl:182
 in eventloop at /home/amit/.julia/v0.4/IJulia/src/IJulia.jl:142
 in anonymous at task.jl:447
while loading In[1], in expression starting on line 1


In [ ]:
using PyPlot

In [ ]:
using QuDynamics
using QuBase

In [ ]:
immutable QuLindbladMasterEqTD <: QuEquation{1}
end

In [ ]:
function fn(H1, H2, t)
    H1 + 9*H2*expm(-(t/5)^2)
end

In [ ]:
N = 2 # Set where to truncate Fock state for cavity
ustate = statevec(1, FiniteBasis(3))
excited = statevec(2, FiniteBasis(3))
ground = statevec(3, FiniteBasis(3))
sigma_ge = tensor(QuArray(eye(N)), ground * excited')  # |g><e|
sigma_ue = tensor(QuArray(eye(N)), ustate * excited')  # |u><e|
a = tensor(lowerop(N), QuArray(eye(3)))
g = 5
H0 = -g * (sigma_ge' * a + a' * sigma_ge)  # time-independent term
H1 = sigma_ue' + sigma_ue

In [ ]:
kappa = 1.5 # Cavity decay rate
gamma = 6  # Atomic decay rate
c_ops = Array(QuBase.QuArray, 3)
c_ops[1] = sqrt(kappa) * a
c_ops[2] = sqrt(5*gamma/9) * sigma_ue
c_ops[3] = sqrt(4*gamma/9) * sigma_ge
# c_ops = [sqrt(kappa) * a, sqrt(5*gamma/9) * sigma_ue, sqrt(4*gamma/9) * sigma_ge]
c_ops

In [ ]:
function QuDynamics.operator(qu_eq::QuLindbladMasterEqTD, t)
    QuDynamics.lindblad_op(fn(H0, H1, t),c_ops)
end

In [ ]:
psi0 = tensor(statevec(1, FiniteBasis(N)), ustate)

In [ ]:
tlist = linspace(-15,15,100)

In [ ]:
quprop = QuPropagator(QuLindbladMasterEqTD(), psi0*psi0', tlist, QuExpmV())

In [ ]:
for (t, rho) in quprop
    println(t, rho)
end

In [ ]:
q = Array[]
push!(q, coeffs(vec(psi0*psi0')))
for (t, psi) in quprop
    push!(q, coeffs(vec(psi)))
end
q

In [ ]:
for i in 1:length(q)
    for j in 1:length(q[i])
    plot(tlist[i], real(q[i][j]), "ro")
    # println(tlist[i], real(q[i]))
    end
end

In [ ]:
for i in 1:length(q)
    for j in 1:length(q[i])
        plot(tlist[i], imag(q[i][j]), "ro")
    # println(tlist[i], real(q[i]))
    end
end